<a href="https://colab.research.google.com/github/regup2000/DLIITKAI/blob/main/Stackoverflow_Tag_Mapv6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

# Plotting Libs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import collections
import nltk
import time
nltk.download('punkt')

# magic function
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#To calculate duration of process
sTime = time.time()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_tags = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/Tags.csv',encoding='iso-8859-1')

In [ ]:
df_tags.shape

(3750994, 2)

In [ ]:
df_tags.tail(10)

,Id,Tag
3750984,40143210,php
3750985,40143210,.htaccess
3750986,40143300,google-bigquery
3750987,40143340,android
3750988,40143340,android-studio
3750989,40143360,javascript
3750990,40143360,vue.js
3750991,40143380,npm
3750992,40143380,mocha
3750993,40143380,babel


In [ ]:
topTen = list(df_tags['Tag'].value_counts().head(10).index)
topTen

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [ ]:
tag_topTen = df_tags[df_tags['Tag'].isin(topTen)]
tag_topTen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826739 entries, 14 to 3750989
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Id      826739 non-null  int64 
 1   Tag     826739 non-null  object
dtypes: int64(1), object(1)
memory usage: 18.9+ MB


In [ ]:
tag_topTen

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#
...,...,...
3750974,40143150,javascript
3750981,40143190,python
3750984,40143210,php
3750987,40143340,android


In [ ]:
def add_tags(question_id):
    return tag_topTen[tag_topTen['Id'] == question_id['Id']].Tag.values

In [ ]:
topTen = tag_topTen.apply(add_tags, axis=1)

In [ ]:
topTen.info()

<class 'pandas.core.series.Series'>
Int64Index: 826739 entries, 14 to 3750989
Series name: None
Non-Null Count   Dtype 
--------------   ----- 
826739 non-null  object
dtypes: object(1)
memory usage: 12.6+ MB


In [ ]:
tag_topTen.head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [ ]:
tag_topTen=pd.concat([tag_topTen, topTen.rename('Tags')], axis=1)
tag_topTen.head()

,Id,Tag,Tags
14,260,c#,[c#]
18,330,c++,[c++]
28,650,c#,[c#]
35,930,c#,[c#]
39,1010,c#,[c#]


In [ ]:
tag_topTen.drop(["Tag"], axis=1, inplace=True)
tag_topTen.shape

(826739, 2)

In [ ]:
tag_topTen.head()

,Id,Tags
14,260,[c#]
18,330,[c++]
28,650,[c#]
35,930,[c#]
39,1010,[c#]


In [ ]:
tag_topTen=tag_topTen.loc[tag_topTen.astype(str).drop_duplicates().index]

In [ ]:
tag_topTen.head()

,Id,Tags
14,260,[c#]
18,330,[c++]
28,650,[c#]
35,930,[c#]
39,1010,[c#]


In [ ]:
#Cleaning questions
question = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/Questions.csv',encoding='iso-8859-1',nrows=25285) #Used partial data to reduce system running time


In [ ]:
question.shape[0]*.4

10114.0

In [ ]:
question = question[['Id','Title','Body']]

In [ ]:
question.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25285 entries, 0 to 25284
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      25285 non-null  int64 
 1   Title   25285 non-null  object
 2   Body    25285 non-null  object
dtypes: int64(1), object(2)
memory usage: 592.7+ KB


In [ ]:
question.head(10)

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
5,330,Should I use nested classes in this case?,<p>I am working on a collection of classes use...
6,470,Homegrown consumption of web services,<p>I've been writing a few web services for a ...
7,580,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...
8,650,Automatically update version number,<p>I would like the version property of my app...
9,810,Visual Studio Setup Project - Per User Registr...,<p>I'm trying to maintain a Setup Project in <...


In [ ]:
import re
def remHTMLElements(text):
    regex = re.compile('<.*?>')
    return re.sub(regex, '', text)

In [ ]:
question['Body'] = question['Body'].apply(remHTMLElements)

In [ ]:
question.head(10)

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...
1,90,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...
2,120,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...
3,180,Function for creating color wheels,This is something I've pseudo-solved many time...
4,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...
5,330,Should I use nested classes in this case?,I am working on a collection of classes used f...
6,470,Homegrown consumption of web services,I've been writing a few web services for a .ne...
7,580,Deploying SQL Server Databases from Test to Live,I wonder how you guys manage deployment of a d...
8,650,Automatically update version number,I would like the version property of my applic...
9,810,Visual Studio Setup Project - Per User Registr...,I'm trying to maintain a Setup Project in Visu...


In [ ]:
question.shape

(25285, 3)

In [ ]:
cleanedData=pd.merge(question, tag_topTen, on='Id')
print(cleanedData.shape)

(10619, 4)


In [ ]:
cleanedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10619 entries, 0 to 10618
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      10619 non-null  int64 
 1   Title   10619 non-null  object
 2   Body    10619 non-null  object
 3   Tags    10619 non-null  object
dtypes: int64(1), object(3)
memory usage: 414.8+ KB


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU,concatenate
from keras.models import Model

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(cleanedData.Tags)
labels = multilabel_binarizer.classes_
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [ ]:
cleanedData['Tags']

0          [c#]
1         [c++]
2          [c#]
3          [c#]
4          [c#]
          ...  
10614      [c#]
10615      [c#]
10616      [c#]
10617     [php]
10618    [java]
Name: Tags, Length: 10619, dtype: object

In [ ]:
cleanedData.shape

(10619, 4)

In [ ]:
train,test=train_test_split(cleanedData,test_size=0.25,random_state=24)

In [ ]:
train.shape,test.shape

((7964, 4), (2655, 4))

In [ ]:
train.head()

,Id,Title,Body,Tags
1560,304800,"Is there a free (LGPL, BSD, etc) implementatio...",I need a JComponent thad enables editing xml d...,[java]
5388,802650,C#: Crash on ManualResetEvent,I wrote my code using this article at msdn as ...,[c#]
2213,405800,Large amount of data - what is the best way to...,we have this scenario:\n\nA server which conta...,[java]
5355,797720,How to convert C# code to object code?,"Hai guys,\n Can any help me how to co...",[c#]
647,154060,Overriding namespaces in gSOAP,I am using gSOAP as a Web Service toolkit and ...,[c++]


In [ ]:
X_train_Title=train['Title']
X_train_Body=train['Body']
y_train=multilabel_binarizer.transform(train['Tags'])
X_test_Title=test['Title']
X_test_Body=test['Body']
y_test=multilabel_binarizer.transform(test['Tags'])

In [ ]:
X_train_Title.head()

1560    Is there a free (LGPL, BSD, etc) implementatio...
5388                        C#: Crash on ManualResetEvent
2213    Large amount of data - what is the best way to...
5355               How to convert C# code to object code?
647                        Overriding namespaces in gSOAP
Name: Title, dtype: object

In [ ]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
train['Title']

1560     Is there a free (LGPL, BSD, etc) implementatio...
5388                         C#: Crash on ManualResetEvent
2213     Large amount of data - what is the best way to...
5355                How to convert C# code to object code?
647                         Overriding namespaces in gSOAP
                               ...                        
5249     How to change my Image DataImageUrlFormatStrin...
10385    Can an STL map iterator go out of bounds throu...
3473                            Avoiding dynamic_cast/RTTI
8535                                  python + Spreadsheet
899                     Multiple Data Tables in PHP/MySQL?
Name: Title, Length: 7964, dtype: object

In [ ]:
len(word_tokenize("Multiple Data Tables in PHP/MySQL?"))

6

In [ ]:
sentLensTitle=[]
for line in train['Title']:
    sentLensTitle.append(len(word_tokenize(line)))
max(sentLensTitle)

33

In [ ]:
maxLenTitleTemp = int(np.quantile(sentLensTitle,0.97))

In [ ]:
maxLenTitleTemp

18

In [ ]:
maxLenTitle = maxLenTitleTemp + (10-(maxLenTitleTemp % 10))

In [ ]:
maxLenTitle

20

In [ ]:
maxLenTitle = 18

In [ ]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_Title)
sequences_train_Title = tok.texts_to_sequences(X_train_Title)

In [ ]:
sequences_train_Title[:10]

[[10, 41, 2, 611, 3470, 3471, 795, 334, 7, 61, 577, 303, 514],
 [6, 796, 20, 3472],
 [233, 1009, 7, 43, 26, 10, 5, 45, 28, 1, 292, 797],
 [4, 1, 108, 6, 33, 1, 39, 33],
 [929, 930, 3, 3473],
 [10, 37, 96, 1, 169, 73, 1256, 12, 2, 74, 3, 6],
 [4, 19, 8, 1257, 44, 2, 1419, 7, 1258, 10, 435, 1654, 164],
 [94, 436, 3474, 181, 436, 11, 1655, 798, 3475],
 [216, 1966, 45, 515],
 [34,
  42,
  54,
  33,
  373,
  11,
  144,
  2458,
  37,
  1967,
  37,
  98,
  30,
  17,
  360,
  141,
  7,
  1259]]

In [ ]:
vocab_lengthTitle=len(tok.index_word.keys())
vocab_lengthTitle

7022

In [ ]:
sequences_matrix_train_Title = sequence.pad_sequences(sequences_train_Title,maxlen=maxLenTitle)
sequences_matrix_train_Title

array([[   0,    0,    0, ...,  577,  303,  514],
       [   0,    0,    0, ...,  796,   20, 3472],
       [   0,    0,    0, ...,    1,  292,  797],
       ...,
       [   0,    0,    0, ...,  175,  731, 7022],
       [   0,    0,    0, ...,    0,   22, 1649],
       [   0,    0,    0, ...,    3,   18,  101]], dtype=int32)

In [ ]:
sequences_matrix_train_Title[:10]

array([[   0,    0,    0,    0,    0,   10,   41,    2,  611, 3470, 3471,
         795,  334,    7,   61,  577,  303,  514],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    6,  796,   20, 3472],
       [   0,    0,    0,    0,    0,    0,  233, 1009,    7,   43,   26,
          10,    5,   45,   28,    1,  292,  797],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    4,
           1,  108,    6,   33,    1,   39,   33],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  929,  930,    3, 3473],
       [   0,    0,    0,    0,    0,    0,   10,   37,   96,    1,  169,
          73, 1256,   12,    2,   74,    3,    6],
       [   0,    0,    0,    0,    0,    4,   19,    8, 1257,   44,    2,
        1419,    7, 1258,   10,  435, 1654,  164],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,   94,  436,
        3474,  181,  436,   11, 1655,  798, 3475],


In [ ]:
sequences_test_Title = tok.texts_to_sequences(X_test_Title)
sequences_matrix_test_Title = sequence.pad_sequences(sequences_test_Title,maxlen=maxLenTitle)

In [ ]:
sequences_matrix_train_Title.shape,sequences_matrix_test_Title.shape,y_train.shape,y_test.shape

((7964, 18), (2655, 18), (7964, 10), (2655, 10))

In [ ]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
sentLensTitleBody=[]
for line in train['Body']:
    sentLensTitleBody.append(len(word_tokenize(line)))
max(sentLensTitleBody)

6589

In [ ]:
maxLenBodyTemp = int(np.quantile(sentLensTitleBody,0.90))

In [ ]:
maxLenBodyTemp

350

In [ ]:
maxLenBodyTemp + (10-(maxLenBodyTemp % 10))

360

In [ ]:
maxLenBody = maxLenBodyTemp + (10-(maxLenBodyTemp % 10))
maxLenBody = 600

In [ ]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_Body)
sequences_train_Body = tok.texts_to_sequences(X_train_Body)

In [ ]:
vocab_lengthBody =len(tok.index_word.keys())
vocab_lengthBody

43803

In [ ]:
sequences_matrix_train_Body = sequence.pad_sequences(sequences_train_Body,maxlen=maxLenBody)
sequences_matrix_train_Body

array([[    0,     0,     0, ...,   709,     9,   498],
       [    0,     0,     0, ...,  6437, 24387,   463],
       [    0,     0,     0, ...,     9,   498, 24388],
       ...,
       [    0,     0,     0, ...,   292,   122, 43802],
       [    0,     0,     0, ...,   107,     9,   498],
       [    0,     0,     0, ...,    97,    37,    11]], dtype=int32)

In [ ]:
sequences_test_Body = tok.texts_to_sequences(X_test_Body)
sequences_matrix_test_Body = sequence.pad_sequences(sequences_test_Body,maxlen=maxLenBody)

In [ ]:
sequences_matrix_train_Title.shape,sequences_matrix_train_Body.shape,y_train.shape

((7964, 18), (7964, 600), (7964, 10))

In [ ]:
sequences_matrix_test_Title.shape,sequences_matrix_test_Body.shape,y_test.shape

((2655, 18), (2655, 600), (2655, 10))

In [ ]:
def RNN():
    # Title Only
    title_input = Input(name='title_input',shape=[maxLenTitle])
    title_Embed = Embedding(vocab_lengthTitle+1,2000,input_length=maxLenTitle,mask_zero=True,name='title_Embed')(title_input)
    gru_out_t = GRU(300)(title_Embed)
    auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)

    # Body Only
    body_input = Input(name='body_input',shape=[maxLenBody])
    body_Embed = Embedding(vocab_lengthBody+1,170,input_length=maxLenBody,mask_zero=True,name='body_Embed')(body_input)
    gru_out_b = GRU(200)(body_Embed)

    # combined with GRU output
    com = concatenate([gru_out_t, gru_out_b])

    # now the combined data is sent to the dense layers
    dense1 = Dense(400,activation='relu')(com)
    dp1 = Dropout(0.5)(dense1)
    bn = BatchNormalization()(dp1)
    dense2 = Dense(150,activation='relu')(bn)

    main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)
    model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
    return model

In [ ]:
model = RNN()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 title_input (InputLayer)    [(None, 18)]                 0         []                            
                                                                                                  
 body_input (InputLayer)     [(None, 600)]                0         []                            
                                                                                                  
 title_Embed (Embedding)     (None, 18, 2000)             1404600   ['title_input[0][0]']         
                                                          0                                       
                                                                                                  
 body_Embed (Embedding)      (None, 600, 170)             7446680   ['body_input[0][0]']    

In [ ]:
model.compile(optimizer='adam',loss={'main_output': 'categorical_crossentropy', 'aux_output': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [ ]:
results=model.fit({'title_input': sequences_matrix_train_Title, 'body_input': sequences_matrix_train_Body},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': sequences_matrix_test_Title, 'body_input': sequences_matrix_test_Body},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=5, batch_size=800)

Epoch 1/5
10/10 [==============================] - 22s 1s/step - loss: 4.4620 - main_output_loss: 2.0126 - aux_output_loss: 2.4494 - main_output_accuracy: 0.3793 - aux_output_accuracy: 0.3178 - val_loss: 4.6891 - val_main_output_loss: 2.3273 - val_aux_output_loss: 2.3618 - val_main_output_accuracy: 0.4938 - val_aux_output_accuracy: 0.4094
Epoch 2/5
10/10 [==============================] - 12s 1s/step - loss: 3.3858 - main_output_loss: 1.2094 - aux_output_loss: 2.1764 - main_output_accuracy: 0.6634 - aux_output_accuracy: 0.4581 - val_loss: 3.8478 - val_main_output_loss: 1.9036 - val_aux_output_loss: 1.9443 - val_main_output_accuracy: 0.5842 - val_aux_output_accuracy: 0.4746
Epoch 3/5
10/10 [==============================] - 12s 1s/step - loss: 2.5767 - main_output_loss: 0.9503 - aux_output_loss: 1.6264 - main_output_accuracy: 0.7779 - aux_output_accuracy: 0.5463 - val_loss: 3.1471 - val_main_output_loss: 1.5946 - val_aux_output_loss: 1.5525 - val_main_output_accuracy: 0.6301 - val_aux_o

In [ ]:
(predicted_main, predicted_aux)=model.predict({'title_input': sequences_matrix_test_Title, 'body_input': sequences_matrix_test_Body},verbose=1)


83/83 [==============================] - 11s 108ms/step


In [ ]:
from sklearn.metrics import classification_report,f1_score

In [ ]:
print(f1_score(y_test,predicted_main>.55,average='samples'))

0.3419752271164701


In [ ]:
print(classification_report(y_test,predicted_main>.55))

              precision    recall  f1-score   support

           0       0.01      0.77      0.03        22
           1       0.34      1.00      0.50       862
           2       0.13      0.98      0.23       312
           3       0.20      0.51      0.29       180
           4       0.00      0.00      0.00         8
           5       0.20      0.98      0.33       461
           6       0.46      0.68      0.55       326
           7       0.36      0.80      0.50       203
           8       0.25      0.84      0.39       308
           9       0.13      0.94      0.22       233

   micro avg       0.20      0.89      0.33      2915
   macro avg       0.21      0.75      0.30      2915
weighted avg       0.27      0.89      0.40      2915
 samples avg       0.22      0.90      0.34      2915



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test.iloc[3]

Id                                                  436700
Title         Is switching app.config at runtime possible?
Body     Is there a way at runtime to switch out an app...
Tags                                                  [c#]
Name: 2431, dtype: object

In [ ]:
predicted_main[3].round(decimals = 2)

array([0.62, 0.98, 0.94, 0.11, 0.27, 0.84, 0.22, 0.21, 0.43, 0.51],
      dtype=float32)

In [ ]:
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/Projects/01 Multiclass Multilabel prediction For stack overflow Questions/stackoverflow_tags.h5')
# model.save('stackoverflow_tags.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
eTime = time.time()
time_taken = eTime-sTime
print(f'Total time consumed: {time_taken:.0f} minute(s)')

Total time consumed: 811 minute(s)
